In [1]:
import requests, json, time, re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
legal = pd.read_csv("./legal_advice.csv")

In [3]:
unpopular = pd.read_csv("./unpopular_opinion.csv")

In [4]:
data = pd.concat([legal, unpopular], axis = 0)

In [5]:
data.head()

,title,posts,subreddit
0,[Super Awesome Update] Sat down on my friend's...,I never thought this would turn out the way it...,legaladvice
1,(Update) Neighbors child has disabilities and ...,Original post: https://www.reddit.com/r/legala...,legaladvice
2,I think my boyfriend has been drugging me to m...,This is in north Texas. \n\nHey so I must apol...,legaladvice
3,Neighbors assaulted me and took my daughter (l...,"Male, late 20’s, CO\n\nI quit drinking soda an...",legaladvice
4,"I told my math teacher about my mother, and sh...",I got my 13 year old brother after school yest...,legaladvice


In [6]:
data['text'] = data['title'] + data['posts']

In [7]:
data.head()

,title,posts,subreddit,text
0,[Super Awesome Update] Sat down on my friend's...,I never thought this would turn out the way it...,legaladvice,[Super Awesome Update] Sat down on my friend's...
1,(Update) Neighbors child has disabilities and ...,Original post: https://www.reddit.com/r/legala...,legaladvice,(Update) Neighbors child has disabilities and ...
2,I think my boyfriend has been drugging me to m...,This is in north Texas. \n\nHey so I must apol...,legaladvice,I think my boyfriend has been drugging me to m...
3,Neighbors assaulted me and took my daughter (l...,"Male, late 20’s, CO\n\nI quit drinking soda an...",legaladvice,Neighbors assaulted me and took my daughter (l...
4,"I told my math teacher about my mother, and sh...",I got my 13 year old brother after school yest...,legaladvice,"I told my math teacher about my mother, and sh..."


In [8]:
data['subreddit'].replace('legaladvice', 1, inplace=True)
data['subreddit'].replace('unpopularopinion', 0, inplace=True)

In [9]:
data.head()

,title,posts,subreddit,text
0,[Super Awesome Update] Sat down on my friend's...,I never thought this would turn out the way it...,1,[Super Awesome Update] Sat down on my friend's...
1,(Update) Neighbors child has disabilities and ...,Original post: https://www.reddit.com/r/legala...,1,(Update) Neighbors child has disabilities and ...
2,I think my boyfriend has been drugging me to m...,This is in north Texas. \n\nHey so I must apol...,1,I think my boyfriend has been drugging me to m...
3,Neighbors assaulted me and took my daughter (l...,"Male, late 20’s, CO\n\nI quit drinking soda an...",1,Neighbors assaulted me and took my daughter (l...
4,"I told my math teacher about my mother, and sh...",I got my 13 year old brother after school yest...,1,"I told my math teacher about my mother, and sh..."


In [10]:
data.shape

(2054, 4)

In [11]:
data.drop_duplicates(inplace=True)

In [12]:
data.shape

(1829, 4)

In [13]:
data.isnull().sum()

title        0
posts        7
subreddit    0
text         7
dtype: int64

In [14]:
data[data['text'].isnull()]

,title,posts,subreddit,text
11,Deadbeat dad ditched my mom when she was pregn...,NaN,1,NaN
83,NY- I sent a rent check to landlord who then c...,NaN,1,NaN
206,I work for a privately owned brewery/restauran...,NaN,1,NaN
304,Police Officer: I am going to search your vehi...,NaN,1,NaN
459,i’m 16 and from england my grandad passed and ...,NaN,1,NaN
592,FL: My schizophrenic neighbor has over 30 time...,NaN,1,NaN
643,"TN: Work overpaid me ($15,000 instead of $1500...",NaN,1,NaN


In [15]:
data.replace(np.nan, "", inplace = True)

In [16]:
data[data['text'].isnull()]

,title,posts,subreddit,text


In [17]:
data['subreddit'].value_counts(normalize=True) # my benchmark (53%)

1    0.533625
0    0.466375
Name: subreddit, dtype: float64

In [38]:
data['text'] = data.posts.map(lambda x: re.sub('\s[\/]?r\/[^s]+', ' ', x)) # getting rid of slashes and spaces
data['text'] = data.posts.map(lambda x: re.sub('http[s]?:\/\/[^\s]*', ' ', x)) # getting rid of url
data['text'] = data.posts.map(lambda x: re.sub('(legal|advice)[s]?', ' ', x,  flags=re.I))
data['text'] = data.posts.map(lambda x: re.sub('(unpopular|opinion)[s]?', ' ', x,  flags=re.I))

In [39]:
data.head()

,title,posts,subreddit,text
0,[Super Awesome Update] Sat down on my friend's...,I never thought this would turn out the way it...,1,I never thought this would turn out the way it...
1,(Update) Neighbors child has disabilities and ...,Original post: \n\nI'm sad about this update ...,1,Original post: \n\nI'm sad about this update ...
2,I think my boyfriend has been drugging me to m...,This is in north Texas. \n\nHey so I must apol...,1,This is in north Texas. \n\nHey so I must apol...
3,Neighbors assaulted me and took my daughter (l...,"Male, late 20’s, CO\n\nI quit drinking soda an...",1,"Male, late 20’s, CO\n\nI quit drinking soda an..."
4,"I told my math teacher about my mother, and sh...",I got my 13 year old brother after school yest...,1,I got my 13 year old brother after school yest...


In [40]:
X = data.drop('subreddit', 1)
y = data['subreddit']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=24)

In [42]:
tfidf = TfidfVectorizer(stop_words='english', max_df=.80, min_df=5) # max features 

In [43]:
train_raw = tfidf.fit_transform(X_train['text'])

In [44]:
train_df = pd.SparseDataFrame(train_raw, columns=tfidf.get_feature_names())

train_df.head()

,00,000,10,100,1000,11,12,13,14,15,...,yell,yelling,yellow,yes,yesterday,york,young,younger,youtube,zero
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.077986
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
train_df.fillna(0, inplace=True)

In [46]:
train_df.isnull().sum()

00           0
000          0
10           0
100          0
1000         0
11           0
12           0
13           0
14           0
15           0
16           0
17           0
18           0
19           0
1st          0
20           0
200          0
2000         0
2014         0
2015         0
2016         0
2017         0
2018         0
2019         0
20s          0
21           0
22           0
23           0
24           0
25           0
            ..
worth        0
wouldn       0
wound        0
wow          0
wrap         0
wrapped      0
wrist        0
write        0
writing      0
written      0
wrong        0
wrote        0
wtf          0
x200b        0
yard         0
yards        0
yea          0
yeah         0
year         0
years        0
yell         0
yelling      0
yellow       0
yes          0
yesterday    0
york         0
young        0
younger      0
youtube      0
zero         0
Length: 3636, dtype: int64

In [47]:
X_test.head()

,title,posts,text
415,"Tittyfucking is Fraud, Plain and Simple",Coffee and popcorn smell better than they tast...,Coffee and popcorn smell better than they tast...
191,Non-Americans bashing America and its citizens...,...especially if their country directly benefi...,...especially if their country directly benefi...
248,[California] I cancelled tattoo appointment be...,I had an appointment for a huge tattoo that wa...,I had an appointment for a huge tattoo that wa...
286,Singers should not be allowed on Talent shows ...,So much good talent is eliminated when matched...,So much good talent is eliminated when matched...
240,I really like traffic,"I get anxiety about death, and car accidents d...","I get anxiety about death, and car accidents d..."


In [53]:
test_raw = tfidf.transform(X_test['text'])
test_df = pd.SparseDataFrame(test_raw, columns=tfidf.get_feature_names())
test_df.fillna(0, inplace=True)
test_df.head()

,00,000,10,100,1000,11,12,13,14,15,...,yell,yelling,yellow,yes,yesterday,york,young,younger,youtube,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
params = {'min_samples_split':[12], 'n_estimators': [1000], 'min_samples_leaf': [3]}

In [55]:
rf = RandomForestClassifier() 

In [56]:
gs = GridSearchCV(rf, param_grid=params, return_train_score=True, cv=5)
gs.fit(train_df, y_train)
gs.score(train_df, y_train) 

0.9828125

In [57]:
gs.score(test_df, y_test)

0.9471766848816029

In [58]:
gs.best_params_ 

{'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 1000}

In [59]:
gs.best_estimator_.feature_importances_

array([2.72649914e-05, 3.76011158e-04, 2.36249297e-04, ...,
       1.93247346e-08, 6.35487020e-06, 7.00308185e-05])